# Gathering event schedule for given year.
 Feel free to input the year in the variable to find the answer. (Let the year be greater than 2018 as fastf1 module doesn't have older data)


In [76]:
import fastf1 as f1
import pandas
year=2018
f1.Cache.enable_cache("C:\movies\platform-tools_r34.0.1-windows")
schedule_data=f1.get_event_schedule(year)
data=schedule_data.loc[schedule_data['RoundNumber']>0].iloc[:,1:3].drop_duplicates().values

print(data)

[['Australia' 'Melbourne']
 ['Bahrain' 'Sakhir']
 ['China' 'Shanghai']
 ['Azerbaijan' 'Baku']
 ['Spain' 'Barcelona']
 ['Monaco' 'Monte Carlo']
 ['Canada' 'Montréal']
 ['France' 'Le Castellet']
 ['Austria' 'Spielberg']
 ['Great Britain' 'Silverstone']
 ['Germany' 'Hockenheim']
 ['Hungary' 'Budapest']
 ['Belgium' 'Spa-Francorchamps']
 ['Italy' 'Monza']
 ['Singapore' 'Singapore']
 ['Russia' 'Sochi']
 ['Japan' 'Suzuka']
 ['United States' 'Austin']
 ['Mexico' 'Mexico City']
 ['Brazil' 'São Paulo']
 ['United Arab Emirates' 'Yas Marina']]


# To gather all the co-ordinate of the locations in the schedule
Give some time for the code to process

In [78]:
from geopy.geocoders import Nominatim
locator=Nominatim(user_agent='my_app')
location_data={}
for i in data:
    
    if i[0]!='Great Britain':
        location=locator.geocode(i[1]+","+i[0])
    else:
        location=locator.geocode(i[1]+","+'England')
    location_data[i[1]+","+i[0]]=(location.latitude,location.longitude)



In [79]:
print(location_data)

{'Melbourne,Australia': (-37.8142176, 144.9631608), 'Sakhir,Bahrain': (26.03169845, 50.51452074628078), 'Shanghai,China': (31.2322758, 121.4692071), 'Baku,Azerbaijan': (40.3755885, 49.8328009), 'Barcelona,Spain': (41.3828939, 2.1774322), 'Monte Carlo,Monaco': (43.7402961, 7.426559), 'Montréal,Canada': (45.5031824, -73.5698065), 'Le Castellet,France': (43.9383814, 5.9789367), 'Spielberg,Austria': (48.2274518, 15.3572755), 'Silverstone,Great Britain': (52.0877287, -1.0241177), 'Hockenheim,Germany': (49.3188892, 8.5475467), 'Budapest,Hungary': (47.48138955, 19.14609412691246), 'Spa-Francorchamps,Belgium': (50.4367166, 5.970040310059851), 'Monza,Italy': (45.639544, 9.278869369225038), 'Singapore,Singapore': (1.357107, 103.8194992), 'Sochi,Russia': (43.5854823, 39.723109), 'Suzuka,Japan': (34.8817102, 136.5836516), 'Austin,United States': (30.2711286, -97.7436995), 'Mexico City,Mexico': (19.4326296, -99.1331785), 'São Paulo,Brazil': (-23.5506507, -46.6333824), 'Yas Marina,United Arab Emirat

# Finding distance between locations
 Using haversine formula to find the shortest distance between two co-ordinates on the map

In [80]:
import math
def cal_distance(cord1,cord2):
    cord1=[cord1[0]*math.pi/180.0,cord1[1]*math.pi/180.0]
    cord2=[cord2[0]*math.pi/180.0,cord2[1]*math.pi/180.0]
    dlat=cord1[0]-cord2[0]
    dlong=cord1[1]-cord2[1]
    def haversin(l):
        return math.pow(math.sin(l/2),2)
    a=(haversin(dlat))+(math.cos(cord1[0])*(math.cos(cord2[0]))*haversin(dlong))
    c=2*math.atan2(math.sqrt(a),math.sqrt(1-a))
    d=6371*c
    return int(d)

    

# Map of given schedule


In [81]:
import folium
import folium.plugins as plugins
numb=1
m=folium.Map(location=[0.0,0.0],zoom_start=2,tiles='Stamen Toner')
for i in location_data.keys():
    folium.Marker(location_data[i],popup=("<i>"+i+"</i>"),icon=plugins.BeautifyIcon(icon='arrow_down',icon_shape='marker',number=numb)).add_to(m)
    numb+=1
trail_cord=list(location_data.values())
total_dist=0
for i in range(0,len(trail_cord)-1):
    line=folium.PolyLine([trail_cord[i],trail_cord[i+1]]).add_to(m)
    attr = {'fill': 'red', 'font-weight': 'bold', 'font-size': '14'}
    dist=cal_distance(trail_cord[i],trail_cord[i+1])
    total_dist+=dist
    plugins.PolyLineTextPath(line,str(dist),attributes=attr,center=True,offset=13).add_to(m)

m

# Total distance travelled 

In [82]:
total_dist

104749

# Using algorithm to reduce total distance travelled

In [83]:
locations=list(location_data.keys())
a={}
for i in range(len(locations)-1):
    for j in range(i+1,len(locations)):
        distance=cal_distance(location_data[locations[i]],location_data[locations[j]])
        a[distance]=[locations[i],locations[j]]





In [84]:
parents={}
for i in locations:
    parents[i]=i

    

In [85]:
dist_matrix=list(a.keys())
dist_matrix.sort()


In [86]:
def find_parent(parents,node):
    while parents[node]!=node:
        node=parents[node]
    return node


# Map of the optimized route

In [87]:
new_distance=0
new_map=folium.Map(location=[0.0,0.0],zoom_start=2,tiles='Stamen Toner')

for i in dist_matrix:
    nodes=a[i]
    parent_a=find_parent(parents,nodes[0])
    parent_b=find_parent(parents,nodes[1])
    
    if parent_b!=parent_a:
        #print(nodes,parent_a,parent_b)
        new_distance+=i
        parents[parent_a]=parent_b
        folium.PolyLine([location_data[nodes[0]],location_data[nodes[1]]]).add_to(new_map)
        
        
        folium.Marker(location_data[nodes[1]],popup="<i>"+nodes[1]+"</i>").add_to(new_map)
        folium.Marker(location_data[nodes[0]],popup="<i>"+nodes[0]+"</i>").add_to(new_map)
        #print(i)
new_map
        

# New distance travelled

In [88]:
print(new_distance)

41043


# Total distance that can be reduced

In [89]:
print((total_dist-new_distance)/total_dist)

0.6081776437006559


C:\Users\suhas\python stuff
